In [1]:
import boto3
import os
import cv2
import numpy as np
from io import BytesIO
from PIL import Image

In [2]:
# s3 클라이언트 생성
s3 = boto3.client('s3')

source_bucket = "smwu-cv-data"  # 원본 데이터 버킷
source_folder = "data/다이캐스팅/g2/" # S3에서 읽어올 폴더 경로

output_bucket = "smwu-cv-team-2-s3"  # 처리된 데이터를 저장할 버킷
output_folder = "preprocessed-data/HighPass/" # 저장할 폴더 경로

In [3]:
# s3에서 원본 이미지 가져오기
def list_images(bucket, prefix):
    
    image_keys = []  # 이미지 키 저장할 리스트
    response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)  # prefix로 시작하는 객체 목록 요청
    
    while response:
        if 'Contents' in response:
            # 이미지 파일만 필터링
            image_keys.extend(
                [item['Key'] for item in response['Contents'] if item['Key'].lower().endswith(('png', 'jpg', 'jpeg'))]
            )
        # 다음 페이지가 있는 경우 계속 요청
        if response.get('IsTruncated'):  # 더 많은 결과가 있는 경우 다음 페이지 요청
            continuation_token = response['NextContinuationToken']
            # 다음 페이지의 객체 목록 요청
            response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix, ContinuationToken=continuation_token)
        else:
            break
    return image_keys

In [4]:
# high pass 필터링 방법 중 라플라시안 필터 적용
def apply_high_pass(image, kernel_size=3, ddepth=cv2.CV_64F):
    # 커널 사이즈 홀수만 적용

    # 이미지를 OpenCV 형식으로 변환
    img_cv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    
    # 라플라시안 필터 적용
    gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)
    laplacian = cv2.Laplacian(gray, ddepth, ksize=kernel_size)

    # 0~255 범위로 변환 후 uint8로 변환
    abs_laplacian = cv2.convertScaleAbs(laplacian)

    # 컬러 이미지로 변환 (저장용)
    processed_img = cv2.cvtColor(abs_laplacian, cv2.COLOR_GRAY2RGB)
    return Image.fromarray(processed_img)

In [7]:
# 이미지 읽고 필터 적용 후 저장
def prepro_and_upload(source_bucket, source_folder, output_bucket, output_folder, kernel_size=3, ddepth=cv2.CV_64F):
    
    image_keys = list_images(source_bucket, source_folder)

    for key in image_keys:
        # S3에서 이미지 다운로드
        response = s3.get_object(Bucket=source_bucket, Key=key)
        img = Image.open(BytesIO(response['Body'].read()))

        # high-pass 필터 적용
        processed_img = apply_high_pass(img)

        # 이미지 저장 준비
        buffer = BytesIO()
        processed_img.save(buffer, format='png')  # png로 저장
        buffer.seek(0)

        # S3에 업로드, 기존 폴더 경로 유지
        kernel_folder = f"{output_folder}kernel_size_{kernel_size}/"
        output_key = kernel_folder + os.path.relpath(key, source_folder)  # 원본 파일 이름 유지
        s3.put_object(Bucket=output_bucket, Key=output_key, Body=buffer)

        print(f"Processed and uploaded: {output_key}")

In [ ]:
# 커널 사이즈 = 1
prepro_and_upload(source_bucket, source_folder, output_bucket, output_folder, kernel_size=1, ddepth=cv2.CV_64F)

Processed and uploaded: preprocessed-data/HighPass/kernel_size_1/dataset_2023-10-17/NG/1/20231013-202440121.png
Processed and uploaded: preprocessed-data/HighPass/kernel_size_1/dataset_2023-10-17/NG/1/20231013-204432370.png
Processed and uploaded: preprocessed-data/HighPass/kernel_size_1/dataset_2023-10-17/NG/1/20231013-210521050.png
Processed and uploaded: preprocessed-data/HighPass/kernel_size_1/dataset_2023-10-17/NG/1/20231013-210600740.png
Processed and uploaded: preprocessed-data/HighPass/kernel_size_1/dataset_2023-10-17/NG/1/20231013-210621893.png
Processed and uploaded: preprocessed-data/HighPass/kernel_size_1/dataset_2023-10-17/NG/1/20231013-210643147.png
Processed and uploaded: preprocessed-data/HighPass/kernel_size_1/dataset_2023-10-17/NG/1/20231013-210704301.png
Processed and uploaded: preprocessed-data/HighPass/kernel_size_1/dataset_2023-10-17/NG/1/20231013-210725454.png
Processed and uploaded: preprocessed-data/HighPass/kernel_size_1/dataset_2023-10-17/NG/1/20231013-210746

In [ ]:
# 커널 사이즈 = 3
prepro_and_upload(source_bucket, source_folder, output_bucket, output_folder, kernel_size=3, ddepth=cv2.CV_64F)

In [ ]:
# 커널 사이즈 = 5
prepro_and_upload(source_bucket, source_folder, output_bucket, output_folder, kernel_size=5, ddepth=cv2.CV_64F)